## Feature Engineering and CV based on 5-th Place Solutions

For all months, deeper trees are better. Just compare the results below with eda_4_33_1

In [1]:
from santander_helper import *

Create all months' data

Load all months' data

In [ ]:
x_train = []
y_train = []
w_train = []
fixed_lag = 6
for i, m in tqdm.tqdm_notebook(enumerate(month_list), total=len(month_list)):
    if m in ['2015-01-28', '2016-06-28']:
        continue
    x_tmp, y_tmp, w_tmp = create_train(m, max_lag=i, fixed_lag=fixed_lag, pattern_flag=True)
    x_train.append(x_tmp)
    y_train.append(y_tmp)
    w_train.append(w_tmp)
del x_tmp, y_tmp, w_tmp
gc.collect()

# Prepare for train and validation
x_val = x_train[-1]
y_val = y_train[-1]
w_val = w_train[-1]

x_train = pd.concat(x_train[:-1], axis=0, ignore_index=True, sort=False)
y_train = pd.concat(y_train[:-1], axis=0, ignore_index=True, sort=False)
w_train = pd.concat(w_train[:-1], axis=0, ignore_index=True, sort=False)

gc.collect()

203

In [ ]:
param = {'objective': 'multi:softprob', 
         'eta': 0.1, 
         'max_depth': 10,
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'merror',
         'min_child_weight': 10,
         'min_split_loss': 1,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}

n_rows = None # number of rows in train dataset, to simplify testing, always set to None
n_repeats = 5
n_trees = 100
train = {'x': x_train.iloc[:n_rows, :], 'y': y_train.iloc[:n_rows], 'w': w_train.iloc[:n_rows]}
val = {'x': x_val.iloc[:n_rows, :], 'y': y_val.iloc[:n_rows], 'w': w_val.iloc[:n_rows]}
df, clfs, running_time = cv_all_month(param, train, val, n_features=350, num_boost_round=n_trees, 
    n_repeats=n_repeats, random_state=0, verbose_eval=True)

[0]	train-merror:0.261372	val-merror:0.280895	train-MAP@7:0	val-MAP@7:0.893906
[1]	train-merror:0.257111	val-merror:0.275118	train-MAP@7:0	val-MAP@7:0.89982
[2]	train-merror:0.254575	val-merror:0.272422	train-MAP@7:0	val-MAP@7:0.902363
[3]	train-merror:0.253303	val-merror:0.27156	train-MAP@7:0	val-MAP@7:0.903421
[4]	train-merror:0.252443	val-merror:0.271376	train-MAP@7:0	val-MAP@7:0.904222
[5]	train-merror:0.251706	val-merror:0.270898	train-MAP@7:0	val-MAP@7:0.904928
[6]	train-merror:0.251122	val-merror:0.270422	train-MAP@7:0	val-MAP@7:0.905355
[7]	train-merror:0.250526	val-merror:0.269961	train-MAP@7:0	val-MAP@7:0.905956
[8]	train-merror:0.250252	val-merror:0.269655	train-MAP@7:0	val-MAP@7:0.906238
[9]	train-merror:0.249577	val-merror:0.269234	train-MAP@7:0	val-MAP@7:0.906717
[10]	train-merror:0.249202	val-merror:0.269208	train-MAP@7:0	val-MAP@7:0.906805
[11]	train-merror:0.248813	val-merror:0.268972	train-MAP@7:0	val-MAP@7:0.907019
[12]	train-merror:0.248341	val-merror:0.268948	train